# Modelling Spatial Deer Distribution using Python

This notebook contains **Python code** and exercises associated with concepts discussed in lecture and [Millington _et al._ (2010)](http://dx.doi.org/10.1016/j.foreco.2009.12). The notebook _Modelling Spatial Deer Distribution using R_ provides the same examples but using R code. Compare the two notebooks to understand how the languages differ.

In [ ]:
import pandas as pd               #for data
import seaborn as sns             #for plotting
import matplotlib.pyplot as plt   #for plotting
import statsmodels.api as sm      #for regression modelling
from scipy import stats           #for kendal tau
from sklearn import linear_model  #for cross-validation
from sklearn import model_selection
import numpy as np                #for mean and variance 
import numpy.ma as ma             #for masked array
import rasterio                   #for reading raster data
from rasterio.plot import show    #for plotting raster data

## Section 1: Loading and Checking Data

In [ ]:
LOI200 = pd.read_csv("data/LOI200.csv") #read the file to a data.frame (assumes data are in data directory)

Now view the first few lines of the data:

In [ ]:
LOI200.head()                                 #view the first part of the data (to check it read properly)

And see a summary of the variables in the data file:

In [ ]:
LOI200.describe()                              #view a summary of the data

Then we use some simple plotting functions to visualise the data:

In [ ]:
#matrix scatter plot for 2nd to 6th columns (python is 0 indexed!)
#no built-in function so use seaborn 
sns.pairplot(LOI200.iloc[:,1:6])    

In [ ]:
#seaborn scatter with axis labels
sns.scatterplot(data=LOI200, x='DistanceLC', y='logDD')
plt.xlabel("Distance LC (km)")
plt.ylabel("log(Deer Density)")        

## Section 2: Correlation and Simple Linear Regression

Now that we have familiarised ourselves with the data, we can do some simple correlations to examine the relationship of deer density with the distance to the nearest lowland conifer stand.  

In [ ]:
print(LOI200[['DistanceLC','logDD']].corr(method='pearson')) #calculate pearson correlation coefficient (r)
print(LOI200[['DistanceLC','logDD']].corr(method='kendall')) #calculate kendall's correlation coefficient (tau)

We find some weak correlations (pandas does not give us an indication of statistical significance). 

The next piece of analysis we will do is to fit simple linear regression models to predict log(deer density) from environmental co-variates.

We could use `linear_model` from the `sklearn` package to fit our simple linear regression:

In [ ]:
df_X = LOI200['DistanceLC']
df_y = LOI200['logDD']                  

new_df_X = df_X.values
new_df_X = new_df_X.reshape(-1,1)

#fit the model 
mod_dlc  = linear_model.LinearRegression()
mod_dlc .fit(new_df_X, df_y)

mod_dlc

However, there is no nice 'summary' method for `sklearn` `linear_model`s as there is in R. We can access model coefficients and intercept directly:

In [ ]:
print(mod_dlc.intercept_)
print(mod_dlc.coef_)

But really this approach seems more suited to working with other computational tools for things like cross-validation (as we'll see below). For more straight-forward model fitting and analysis, we can use functions available in the `statsmodels` package. Read the docs and more detail [here](https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html). Specifically we can use the `OLS` function from the `statsmodels.api`. 

Using the `OLS` function from `statsmodels` to fit a regression requires we create an `OLS` object first, then use the `fit` [method](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.fit.html) on that object to create a `RegressionResults` [object](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults). An easy way to create an `OLS` object is to use the `from_formula` [method](https://www.statsmodels.org/dev/dev/generated/statsmodels.base.model.Model.from_formula.html?highlight=from_formula#statsmodels.base.model.Model.from_formula) to pass the equation of the model we want to fit (as well as indicating what the data are that we are using):

In [ ]:
#requires import statsmodels.api as sm
mod_dlc = sm.OLS.from_formula("logDD ~ DistanceLC", data = LOI200) 
mod_dlc_RR = mod_dlc.fit()
print(mod_dlc_RR.summary())

Let's see how this model looks using a scatter plot of the variables - seaborn will fit a regression with confidence envelope for us (but note we have less control than in R)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
g = sns.regplot(x=LOI200.DistanceLC, y=LOI200.logDD) 

From the [statsmodel docs](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html) we can see that we can access the model residuals using `.resid`. We can use this to plot our own histogram of model residuals, for example:

In [ ]:
mod_dlc_RR.resid.hist(bins=8)           

This plot shows that the residuals are reasonably normally distributed and that we are likely not violating the assumptions of simple linear regression. 

R provides built-in diagnostic plotting functions for regression model objects. `statsmodels` in python doesn't offer this, but there are [relatively straight-forward ways to run diagnostics](https://www.statsmodels.org/dev/examples/notebooks/generated/regression_diagnostics.html).   

To get some other useful model outputs, we need to get items from the model summary:

In [ ]:
round(mod_dlc_RR.params,3)    #directly access the model coefficients 

In [ ]:
round(mod_dlc_RR.rsquared,2)  #directly access the r.squared of the model

In [ ]:
round(mod_dlc_RR.pvalues,3)

The code in this section has provided all we need to know to calculate the values in the first row of Table 1 in Millington et al. (2010).

### Task 1.
Use the code above to help you calculate values for the four other variables in Table 1 that have p < 0.1. Check you can get values from your code that correspond to those in Table 1 of Millington _et al._ (2010) 

## Section 3. Multiple linear regression models
Fitting linear regression models with more than one are almost as straighforward as for simple (univariate) linear regression models. We just need to add the additional variables into the model equation:

In [ ]:
mod_dlc_dbh = sm.OLS.from_formula("logDD ~ DistanceLC + NewDBH", data = LOI200) 
mod_dlc_dbh_RR = mod_dlc_dbh.fit()
print(mod_dlc_dbh_RR.summary())

To get a kendall correlation coefficient we need to output the predicted logDDs and then test using `kendalltau` from the [`stats` module of the `scipy` package](https://docs.scipy.org/doc/scipy/reference/stats.html):

In [ ]:
mod_dlc_dbh_RR_pred  = mod_dlc_dbh_RR.predict()

In [ ]:
#requires from scipy import stats 
stats.kendalltau(LOI200['logDD'], mod_dlc_dbh_RR_pred)

### Task 2.
Use the code above to write your own code to calculate the values in Table 2 of Millington _et al._ (2010) for the Full Model (all values except for cross-validation)

## Section 4. Cross-validation
To complete the bottom two lines of Table 2 in Millington et al. (2010) we need to run cross-validation. In python, we can use functionality from the [`sklearn` package](https://scikit-learn.org/stable/index.html) for [cross-validation](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6).

First, we need to fit the model again using `sklearn` to create a model object that it will be able handle:  

In [ ]:
#requires from sklearn import linear_model

#create independent and dependent variables
df_X = LOI200[['DistanceLC','NewDBH']]  
df_y = LOI200['logDD']                  

#fit the mode 
mod_dlc_dbh = linear_model.LinearRegression()
mod_dlc_dbh.fit(df_X, df_y)

#we can get simple otuputs
print(mod_dlc_dbh.coef_)  #model coefficients
print(mod_dlc_dbh.score(df_X, df_y)) #r2

We can use the fitted model to estimate predicted values for all out observations:

In [ ]:
Predicted = mod_dlc_dbh.predict(df_X)
print(Predicted)

To do a single 5-fold cross validation for the regression model we would use:

In [ ]:
#requires from sklearn import model_selection
cvpred = model_selection.cross_val_predict(mod_dlc_dbh, df_X, df_y, cv=5)
print(cvpred)

This has created an array of predicted values based on models fit for each of the folds created (see `model_selection.KFold` for [more](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html?highlight=kfold#sklearn.model_selection.KFold) on how the folds are created). 

Now append the predictors (from both all observations and the cross-validation) to the original data to allow us to calculate correlations for comparing between using all observed data vs k-folds:

In [ ]:
CVdata = LOI200
CVdata['Predicted'] = Predicted
CVdata['cvpred'] = cvpred

#Predicted
r = CVdata['logDD'].corr(CVdata['Predicted'])
r2 = r**2
t = CVdata['logDD'].corr(CVdata['Predicted'], method='kendall')
    
print("r2 (obs): ", round(r2,3))   
print("tau (obs): ", round(t,3))
print('\n')

#Cross Validated
r = CVdata['logDD'].corr(CVdata['cvpred'])
r2 = r**2
t = CVdata['logDD'].corr(CVdata['cvpred'], method='kendall')
    
print("r2 (cv): ", round(r2,3))   
print("tau (cv): ", round(t,3))

But note how the caption of Table 2 indicates that 
>"estimates for cross-validation results are 95% confidence intervals calculated from mean and variance of 100 repetitions." 

To do this we need to create a loop to run the cross-validation multiple times, storing the results so we can calculate the mean
and variance:

In [ ]:
cv_r2 = []
cv_tau = []

for i in range(100):

    kf = model_selection.KFold(n_splits=5,shuffle=True, random_state=i)  #needed to create random folds (using i)
    cvpred = model_selection.cross_val_predict(mod_dlc_dbh, df_X, df_y, cv=kf)  #predict using current random folds
    
    #correlations
    r = LOI200['logDD'].corr(pd.Series(cvpred))
    tau = LOI200['logDD'].corr(pd.Series(cvpred), method='kendall')
    
    #append to list
    cv_r2.append(r**2)
    cv_tau.append(tau)
  

Now we can calculate mean and variance for r2 and tau

In [ ]:
print(np.mean(cv_r2))
print(np.mean(cv_tau))
print(1.96 * np.var(cv_r2))  
print(1.96 * np.var(cv_tau))

#printoptions precision does not work for numpy.float64! 
#see https://stackoverflow.com/questions/47637972/numpy-set-printoptions-precision-ignored-in-list-of-tuples

### Task 3.

Build on the code above to calculate the cross-validation values for the 'Full' model

## Section 5. Spatial Estimation
Later in Millington _et al._ (2010) models fit from the sample data (at 51 locations) were used to predict deer density across a subsection of the study area. We readily do this in R using the functionality in the `raster` package. Things are not so straight-forward in python because the raster processing tools do not seem to be so well developed... 

In python, the main package for raster tools seems to be [`rasterio`](https://rasterio.readthedocs.io/). However, the rasterio package doesn't have predict methods like for R and to do what we want to do here, [others have suggested simply using R!](https://stackoverflow.com/questions/48853484/how-to-structure-data-for-prediction-using-rasters-as-targets). As its name suggests, rasterio is focused on raster input and output.  

After some exploring, I thought we might be able use [pyimpute](https://github.com/perrygeo/pyimpute) which has some nice features and contains a lot of the functionality we want. _But_, pyimpute assumes the data used to fit models are from the study rasters themselves (indicated by points within the raster). In the current situation, we have already fit the model using point data and now simply want to apply the fitted model parameters to our explanatory rasters.

Further googling suggested we could use `numpy arrays` with the numpy predict methods, for example see [here](https://stackoverflow.com/questions/29036179/what-is-the-python-equivalent-to-r-predict-function-for-linear-models) and [here](https://www.statsmodels.org/devel/examples/notebooks/generated/predict.html). We would have to do some fiddling moving back and forth between 1D arrays and 2D rasters but it should be possible. To specify the point location values we will need to use [numpy masked arrays](https://numpy.org/doc/stable/reference/maskedarray.generic.html) (e.g. see [here](https://stackoverflow.com/a/38856546) and [here](https://stackoverflow.com/a/38194439)).

First, we'll do a uni-variate model for distance to lowland conifer. Let's view the raster data we're going to use to predicte deer distribution (distance to lowland conifer).  

In [ ]:
#read the raster with rasterio,  requires import rasterio and from rasterio.plot import show
dlc_r = rasterio.open('data/LOI200_dlc_km.asc')
rasterio.plot.show(dlc_r)

Note the 'empty' block (no data) in the bottom left. This will be important in a minute...

Next, we create [a 1D `ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) from this 2D raster: 

In [ ]:
#read the raster with rasterio,  requires import rasterio
#we need to use src.read for flatten() method below
with rasterio.open('data/LOI200_dlc_km.asc') as src:
    dlc_r = src.read(1)  
    
dlc_f = dlc_r.flatten()  #flatten from 2d to 1D

#check we have produced a 1D array
print(type(dlc_f))
print(dlc_f.shape)

Now, remember that block of no data in the bottom left of the raster? We tell python to ignore this. We do that using a masked array:

In [ ]:
#create the mask. requres import numpy.ma as ma
ma_dlc = ma.masked_values(dlc_f, -9999)   #specify that -9999 is no data 
mac_dlc = ma.compressed(ma_dlc)           #remove these no data values

We can use the `mod_dlc` sklearn linear model object we created above to make our predictions:

In [ ]:
#fit the model 
mod_dlc  = linear_model.LinearRegression()
mod_dlc .fit(new_df_X, df_y)

res = mod_dlc.predict(dlc_f.reshape(-1,1))
res = res.astype(np.float32) #original were float32

Before we convert our 1D data back to 2D we need re-insert the no data values we removed:

In [ ]:
pred_dlc = ma.MaskedArray(res, mask=ma_dlc.mask,fill_value=-9999)

And now we can convert back to a 2D map:

In [ ]:
pred2D = pred_dlc.reshape(dlc_r.shape)

And plot!

In [ ]:
rasterio.plot.show(pred2D)

Phew! What took a couple of lines in R, took a lot more work in python. And getting a legend on that plot will take [even more messing with matplotlib](https://stackoverflow.com/questions/25482876/how-to-add-legend-to-imshow-in-matplotlib). We can at least just make the plot a little larger...  

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

im = ax.imshow(pred2D)

plt.show()

### Task 4.

Use code in this section to create a predicted deer density map for the 'best model' (i.e. using `LOI200_dlc_km.asc` and `LOI200_Meandbh_cm.asc` as predictor maps).

_Hint:_ to predict a model across multiple `np array`s you will need to [stack](https://stackoverflow.com/a/48818880) the arrays into a single array object (then `predict` on that object). 